## An example script to show data splitting as a part of data preprocessing

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker import get_execution_role
import sagemaker

input_bucket_name ='public-datasets-imaging'
dataset_name = 'mednist'

output_dataset_name = 'mednist-processed'

input_s3_uri = f's3://{input_bucket_name}/{dataset_name}'
output_s3_uri = f's3://{input_bucket_name}/{output_dataset_name}'
sagemaker_local_session = sagemaker.local.LocalSession()
execution_role = 'AmazonSageMaker-ExecutionRole-20240907T181142'



In [ ]:
image_uri='575108919340.dkr.ecr.us-east-2.amazonaws.com/sm-preprocessing:latest'
script_processor = ScriptProcessor(command=['python3'],
                                image_uri=image_uri,
                                role=execution_role,
                                instance_type='ml.c4.xlarge',
                                instance_count=1,
                                ) 

In [ ]:
script_processor.run(code='preprocessing.py',
                     job_name='data-splitting',
                    inputs=[ProcessingInput(source=input_s3_uri, destination='/opt/ml/processing/input')],
                    outputs=[ProcessingOutput(source='/opt/ml/processing/output/train', 
                                              destination=output_s3_uri + '/train'),
                             ProcessingOutput(source='/opt/ml/processing/output/test', 
                                              destination=output_s3_uri + '/test'),
                             ProcessingOutput(source='/opt/ml/processing/output/val', 
                                              destination=output_s3_uri + '/val')],
                    arguments=['--train-split', '0.7', '--test-split', '0.15', '--val-split', '0.15']
                    )
